### NOTE: 
 > Please Specify correct path for each file and output files will be in same directory
 
 > 3 files are expected to run this notebook all at once ('hi_100.txt' ,'cs689_assignment.txt' , 'ground_truth_tokens.txt')
 
 > Restart and Run all the Kernal to execute whole notebook at once
 
 > The function named 'write_to_file_and_print_top_k' writes output of some models also
     print top-k values (based on freq) in current directory, Please comment that if no file is expected
 

# Problem-1 

In [1]:
file_path = 'hi_100.txt'  
try:
    with open(file_path, 'r',encoding='utf-8') as file:
        data = file.readlines()
        print(data[:5])

except FileNotFoundError:
    print(f"The file '{file_path}' was not found.")
except Exception as e:
    print(f"An error occurred: {e}")

['आवेदन करने की आखिरी तारीख 31 जनवरी, 2020 है।\n', 'इतनी दुआ कर दो हमारे लिए कि जितना प्यार दुनिया ने आपको दिया है, बस उतना ही हमें भी मिल जाए|”\n', 'मोदी सरकार के पहले कार्यकाल में भी तीन तलाक को लेकर बिल लाया गया था, हालांकि तब यह राज्यसभा में पास नहीं हो पाया था.\n', 'भाजपा के दिवंगत नेता प्रमोद महाजन की बेटी पूनम महाजन को सचिव बनाया गया है.\n', "ऐसी स्थिति में एक न्यायपूर्ण सरकार सार्वजनिक वित्त का इस तरह इस्तेमाल करती है कि संसाधनों का आवंटन, सभी के उपभोग वाले उत्पादों की व्यवहार्यता और समग्र वृहद-आर्थिक प्रबंधन 'निष्पक्षता के रूप में न्याय' को बढ़ाए।\n"]


### function for Unicode Correction

In [5]:
alphabets = {
    "क": ['क्', 'अ'], "ख": ['ख्', 'अ'], "ग": ['ग्', 'अ'], "घ": ['घ्', 'अ'], "ङ": ['ङ्', 'अ'],
    "च": ['च्', 'अ'], "छ": ['छ्', 'अ'], "ज": ['ज्', 'अ'], "झ": ['झ्', 'अ'], "ञ": ['ञ्', 'अ'],
    "ट": ['ट्', 'अ'], "ठ": ['ठ्', 'अ'], "ड": ['ड्', 'अ'],"ड़":['ड़'] ,"ढ": ['ढ्', 'अ'], "ण": ['ण्', 'अ'],
    "त": ['त्', 'अ'], "थ": ['थ्', 'अ'], "द": ['द्', 'अ'], "ध": ['ध्', 'अ'], "न": ['न्', 'अ'],
    "प": ['प्', 'अ'], "फ": ['फ्', 'अ'], "ब": ['ब्', 'अ'], "भ": ['भ्', 'अ'], "म": ['म्', 'अ'],
    "य": ['य्', 'अ'], "र": ['र्', 'अ'], "ल": ['ल्', 'अ'], "व": ['व्', 'अ'],
    "श": ['श्', 'अ'], "ष": ['ष्', 'अ'], "स": ['स्', 'अ'], "ह": ['ह्', 'अ'],
    "क्ष": ['क्', 'ष्', 'अ'], "त्र": ['त्', 'र्', 'अ'],
    "ज्ञ": ['ज्', 'ञ्', 'अ'],
    "अ": ['अ'], "आ": ['आ'], "इ": ['इ'], "ई": ['ई'], "उ": ['उ'], "ऊ": ['ऊ'],
    "ऋ": ['ऋ'], "ॠ": ['ॠ'], "ऌ": ['ऌ'], "ॡ": ['ॡ'],
    "ए": ['ए'], "ऐ": ['ऐ'], "ओ": ['ओ'], "औ": ['औ'],
    "ा": ['अ'], "ि": ['इ'], "ी": ['ई'], "ु": ['उ'], "ू": ['ऊ'],
    "ृ": ['ऋ'], "ॄ": ['ॠ'], "ॢ": ['ऌ'], "ॣ": ['ॡ'],
    "े": ['ए'], "ै": ['ऐ'],
    "ो": ['ओ'], "ौ": ['औ'],
    "ं": ['अं'], "ः": ['अः'], "ँ": ['अँ'], '१':['१'], '२':['२'], '३':['३'], '४':['४'], '५':['५'], '६':['६'], '७':['७'],
    '८':['८'], '९':['९']
}

reverse_matras = {"अ": "", "आ": "ा", "इ": "ि", "ई": "ी", "उ": "ु", "ऊ": "ू", "ऋ": "ृ", "ॠ": "ॄ", "ऌ": "ॢ", "ॡ": "ॣ",
                  "ए": "े", "ऐ": "ै", "ओ": "ो", "औ": "ौ", "अं": "ं", "अः": "ः", "अँ": "ँ"}
second_matras = {"अं": "ं", "अः": "ः", "अँ": "ँ"}
reverse_matras_second = {"ं": "अं", "ः": "अः", "ँ": "अँ"}
consonant_mapping = {
    "क": "k", "ख": "kh", "ग": "g", "घ": "gh", "ङ": "N",
    "च": "ch", "छ": "chh", "ज": "j", "झ": "jh", "ञ": "~N",
    "ट": "T", "ठ": "Th", "ड": "D", "ढ": "Dh", "ण": "N",
    "त": "t", "थ": "th", "द": "d", "ध": "dh", "न": "n",
    "प": "p", "फ": "ph", "ब": "b", "भ": "bh", "म": "m",
    "य": "y", "र": "r", "ल": "l", "व": "v", "श": "sh",
    "ष": "Sh", "स": "s", "ह": "h", "ळ": "L", "क्ष": "kSh","ड़":"D",
    "ज्ञ": "jyn", "ड़": "R", "य़": "y", "ज़": "z", "ब़": "b", "क़": "q", "ख़": "Kh", "ग़": "G",
    "ड़": "R", "ढ़": "Rh", "फ़": "f", "श़": "sh", "ऋ": "Ri", "ॠ": "Ri", "ॡ": "Li", "ऌ": "Li", "ऴ": "L", "ॐ": "OM",
    "ऽ": "'"
}

matras = {"ा": "आ", "ि": "इ", "ी": "ई", "ु": "उ", "ू": "ऊ", "ृ": "ऋ", "ॄ": "ॠ", "ॢ": "ऌ", "ॣ": "ॡ", "े": "ए", "ै": "ऐ",
          "ो": "ओ", "ौ": "औ", "ं": "अं", "ः": "अः", "ँ": "अँ"}

halanta_char =  {
    "क्": 'क', "ख्": 'ख', "ग्": 'ग', "घ्": 'घ', "ङ्": 'ङ',
    "च्": 'च', "छ्": 'छ', "ज्": 'ज', "झ्": 'झ', "ञ्": 'ञ',
    "ट्": 'ट', "ठ्": 'ठ', "ड्": 'ड', "ढ्": 'ढ', "ण्": 'ण',
    "त्": 'त', "थ्": 'थ', "द्": 'द', "ध्": 'ध', "न्": 'न',
    "प्": 'प', "फ्": 'फ', "ब्": 'ब', "भ्": 'भ', "म्": 'म',
    "य्": 'य', "र्": 'र', "ल्": 'ल', "व्": 'व',
    "श्": 'श', "ष्": 'ष', "स्": 'स', "ह्": 'ह',
    "ळ्": 'ळ', "क्ष्": 'क्ष', "ज्ञ्": 'ज्ञ', "ड़्": 'ड़', "य़्": 'य़',
    "ज़्": 'ज़', "ब़्": 'ब़', "क़्": 'क़', "ख़्": 'ख़', "ग़्": 'ग़',
    "ड़्": 'ड़', "ढ़्": 'ढ़', "फ़्": 'फ़', "श़्": 'श़',
    "ऋ्": 'ऋ', "ॠ्": 'ॠ', "ॡ्": 'ॡ', "ऌ्": 'ऌ', "ऴ्": 'ऴ',
    "ॐ्": 'ॐ', "ऽ्": 'ऽ'
}


In [7]:
def character_conversion(line):
    chars_list = []
    i = 0
        
    drop_list = symbols_list = ['।', ' ', ',', '०', '.', '?', '!', '-', ':', ';']
    while i < len(line):
        current_char = line[i]
        if(current_char in drop_list):
            i+=1
            continue
        next_char = line[i + 1] if i + 1 < len(line) else None
        next_next_char = line[i + 2] if i + 2 < len(line) else None

        if current_char in consonant_mapping and next_char not in matras:
            if current_char in alphabets:
                chars_list.extend(alphabets[current_char])
            else:
                chars_list.extend(current_char)
        elif current_char in matras:
            chars_list.append(matras[current_char])
        else:
            chars_list.extend(current_char)
        i += 1

    chars_list = [char for char in chars_list if char != " "]  # Remove empty characters
    return chars_list

In [8]:
val = character_conversion("सूरज की किरणों ने समुद्र की लहरों को चमकाया")
print(val, "count of characters is: ", len(val))

['स', 'ऊ', 'र्', 'अ', 'ज्', 'अ', 'क', 'ई', 'क', 'इ', 'र्', 'अ', 'ण', 'ओ', 'अं', 'न', 'ए', 'स्', 'अ', 'म', 'उ', 'द्', 'अ', '्', 'र्', 'अ', 'क', 'ई', 'ल्', 'अ', 'ह्', 'अ', 'र', 'ओ', 'अं', 'क', 'ओ', 'च्', 'अ', 'म्', 'अ', 'क', 'आ', 'य', 'आ'] count of characters is:  45


# Problem-2

### Function definations for syllable conversion and collecting top-20 highest freq

In [9]:
def top_freq_unigram(u_freq):
    sorted_unigram_freq = sorted(u_freq.items(), key=lambda x: x[1], reverse=True)
    top_20_unigram = dict(sorted_unigram_freq[:20])
    return top_20_unigram

def top_freq_bigram(b_freq):
    sorted_bigram_freq = sorted(b_freq.items(), key=lambda x: x[1], reverse=True)
    top_20_bigram = dict(sorted_bigram_freq[:20])
    return top_20_bigram


In [10]:
#  This function expects args are chars so dependent on character_conversion inherently
def syllable_convert(val):
    val = character_conversion(val)
    syllables = []
    current_syllable = ""
    i = 0

    while i < len(val):
        if val[i] in consonant_mapping:
            current_syllable += val[i]
        elif val[i] in halanta_char:
            current_syllable += halanta_char[val[i]]
        else:
            if current_syllable != "":
                if val[i] in reverse_matras:
                    current_syllable += reverse_matras[val[i]]
                if i + 1 < len(val) and val[i + 1] in second_matras:
                    current_syllable += second_matras[val[i + 1]]
                    i += 1
            else:
                current_syllable = val[i]
            syllables.append(current_syllable)
            current_syllable = ""
        i += 1

    # Remove empty strings from the list
    syllables = [x for x in syllables if x]

    return syllables

syll = syllable_convert("बारिश के मौसम में, लोग गरम चाय का आनंद लेते हैं।")
print(syll, "count of syllable is: ", len(syll))

['बा', 'रि', 'श', 'के', 'मौ', 'स', 'म', 'में', 'लो', 'ग', 'ग', 'र', 'म', 'चा', 'य', 'का', 'आ', 'नं', 'द', 'ले', 'ते', 'हैं'] count of syllable is:  22


In [11]:
from collections import Counter

processed_data = [(character_conversion(line), syllable_convert(line)) for line in data]

unigram_freq_char = Counter(char for val, _ in processed_data for char in val)
unigram_freq = Counter(syllable for _, freq in processed_data for syllable in freq)

bigram_freq_char = Counter(" ".join((val[i], val[i + 1])) for val, _ in processed_data for i in range(len(val) - 1))
bigram_freq = Counter(" ".join((freq[i], freq[i + 1])) for _, freq in processed_data for i in range(len(freq) - 1))

# Print top 20 values
print("\nSyllable Unigram Frequency:\n")
print(dict(unigram_freq.most_common(20)))

print("\nSyllable Bigram Frequency:\n")
print(dict(bigram_freq.most_common(20)))

print("\nCharacter Unigram Frequency :\n")
print(dict(unigram_freq_char.most_common(20)))

print("\nCharacter Bigram Frequency :\n")
print(dict(bigram_freq_char.most_common(20)))



Syllable Unigram Frequency:

{'र': 1403416, '्': 1385294, 'क': 775832, 'स': 686588, 'न': 599409, 'प': 546428, 'त': 473505, 'के': 399015, 'ल': 390598, 'म': 355766, 'ने': 325171, 'ए': 302617, 'का': 300554, '\n': 298799, 'या': 284731, 'ह': 282565, 'य': 277701, 'ब': 273580, 'ग': 269704, 'में': 259827}

Syllable Bigram Frequency:

{'र ्': 230510, 'क र': 176396, 'स ्': 168036, '् र': 167055, 'क ्': 157814, 'प ्': 141222, 'त ्': 123838, 'औ र': 115698, 'प र': 107416, '् य': 91062, 'इ स': 88005, 'न ्': 83708, 'है \n': 79522, '् या': 69781, 'ड ़': 65370, 'ए क': 65133, 'का र': 63436, '् ट': 59278, 'द ्': 58980, 'ल ्': 57341}

Syllable Unigram Frequency (Character-based):

{'अ': 8522539, 'आ': 2991109, 'ए': 2318442, 'ई': 1460305, 'क': 1442541, 'इ': 1432973, 'र्': 1403772, '्': 1385508, 'अं': 1201207, 'ओ': 896588, 'ह': 848785, 'क्': 777423, 'न': 734863, 'र': 711877, 'म': 697402, 'स्': 686835, 'न्': 599585, 'स': 596873, 'उ': 587149, 'प्': 547147}

Syllable Bigram Frequency (Character-based):

{'र् अ

# Problem-3

In [12]:
# Attempted manually on site "https://bangla.iitk.ac.in/cs689/main"

# Problem-4 

### Reading Data File 'hi_100.txt'

In [13]:
file_path = 'hi_100.txt'  
try:
    with open(file_path, 'r',encoding='utf-8') as file:
        data = file.read()
        print(data[:1000])

except FileNotFoundError:
    print(f"The file '{file_path}' was not found.")
except Exception as e:
    print(f"An error occurred: {e}")

आवेदन करने की आखिरी तारीख 31 जनवरी, 2020 है।
इतनी दुआ कर दो हमारे लिए कि जितना प्यार दुनिया ने आपको दिया है, बस उतना ही हमें भी मिल जाए|”
मोदी सरकार के पहले कार्यकाल में भी तीन तलाक को लेकर बिल लाया गया था, हालांकि तब यह राज्यसभा में पास नहीं हो पाया था.
भाजपा के दिवंगत नेता प्रमोद महाजन की बेटी पूनम महाजन को सचिव बनाया गया है.
ऐसी स्थिति में एक न्यायपूर्ण सरकार सार्वजनिक वित्त का इस तरह इस्तेमाल करती है कि संसाधनों का आवंटन, सभी के उपभोग वाले उत्पादों की व्यवहार्यता और समग्र वृहद-आर्थिक प्रबंधन 'निष्पक्षता के रूप में न्याय' को बढ़ाए।
दिलचस्प है कि डीसीएचएल के चेयरमैन टी वेंकटरमन रेड्डी और वाइस चेयरमैन टी विनायक रवि रेड्डी इस बैठक में मौजूद नहीं थे।
इस आम चुनाव में भाजपा नेता सतीश कुमार गौतम को सबसे अधिक 6 लाख 56 हजार 215 वोट प्राप्त हुए.
आयरलैंड टीम के विकेटकीपर बल्लेबाज नियाल ओ'ब्रायन ने अंतर्राष्ट्रीय क्रिकेट से संन्यास का ऐलान कर दिया है। साल 2002 में डेनमार्क के खिलाफ के एकदिवसीय क्रिकेट में डेब्यू करने वाले ब्रायन ने 36 साल की उम्र में संन्यास का ऐलान किया। नियाल ओ'ब्रायन आयरलैंड

In [14]:
#  Uncomment this line for running model in less time, but models will not be trained on whole corpus
# data = data[:100]

### Function definations for calculating top k freqs and writing them into files 

In [15]:
def write_to_file_and_print_top_k(model_name, vocab, k, freq_arr):
    """
    Write token frequencies to files and print the top k tokens for each gram type.

    Parameters:
    - model_name (str): The name of the model.
    - vocab (str): The vocabulary name.
    - k (int): The number of top tokens to display.
    - freq_arr (tuple): A tuple containing four dictionaries of token frequencies
                       for unigrams, bigrams, syllables, and characters.

    Returns:
    None
    """
    for grams, gram_type in [(freq_arr[i], ["unigram", "bigram", "syllables", "char"][i]) for i in range(4)]:
        sorted_grams = sorted(grams.items(), key=lambda x: x[1], reverse=True)

#         Uncomment below line will write all results in the file with respective names
        with open(f"{model_name}_{gram_type}_{vocab}.txt", 'w', encoding='utf-8') as f:
            f.writelines([f"{gram}: {count}\n" for gram, count in sorted_grams])

        print(f"\nTop {k} For {gram_type}\n")
        print("\n".join([f"{token}: {freq}" for token, freq in sorted_grams[:k]]))


        
def count_frequencies(tokenizer, data):
    """
    Count the frequencies of unigrams, bigrams, syllables, and characters in the given data.

    Parameters:
    - tokenizer (Tokenizer): A tokenizer object to encode the data.
    - data (str): The input data for frequency analysis.

    Returns:
    tuple: A tuple containing four Counter objects representing the frequencies of unigrams, bigrams,
           syllable bigrams, and character bigrams in the given data.
    """
    # generating tokens in data by tokenizer
    tokens = tokenizer.encode(data, out_type=str)
    
    # Count unigram frequencies
    unigram_freqs = Counter(tokens)
    
    # Count bigram frequencies
    bigrams = [(tokens[i], tokens[i + 1]) for i in range(len(tokens) - 1)]
    bigram_freqs = Counter(bigrams)

    # generating syllables and characters of tokens
    syllables = []
    characters = []
    for tok in tokens:
        syllables.extend(syllable_convert(tok))
        characters.extend(character_conversion(tok))
        
    bigram_syllables = [(syllables[i], syllables[i + 1]) for i in range(len(syllables) - 1)]
    bigram_characters = [(characters[i], characters[i + 1]) for i in range(len(characters) - 1)]
    
    syllable_freqs = Counter(bigram_syllables)
    char_freqs = Counter(bigram_characters)

    return unigram_freqs, bigram_freqs, syllable_freqs, char_freqs

### Function definations to return Unigram and BPE tokenizer

In [16]:
import sentencepiece as spm
from collections import Counter

def train_unigram_tokenizer(file_path, vocab_size):
    """
    Train and return a SentencePiece unigram tokenizer using the specified file_path and vocab_size.

    Parameters:
    - file_path (str): The path to the input file for training the tokenizer.
    - vocab_size (int): The desired vocabulary size for the trained tokenizer.

    Returns:
    SentencePieceProcessor: A trained unigram tokenizer.
    """
    # Train unigram tokenizer
    spm.SentencePieceTrainer.Train(input=file_path, model_prefix='unigram_model', model_type='unigram', vocab_size=vocab_size)

    # Load the trained unigram tokenizer
    model_file = 'unigram_model.model'
    unigram_tokenizer = spm.SentencePieceProcessor(model_file)
    return unigram_tokenizer

def train_bpe_tokenizer(file_path, vocab_size):
    """
    Train and return a SentencePiece BPE tokenizer using the specified file_path and vocab_size.

    Parameters:
    - file_path (str): The path to the input file for training the tokenizer.
    - vocab_size (int): The desired vocabulary size for the trained tokenizer.

    Returns:
    SentencePieceProcessor: A trained BPE tokenizer.
    """
    # Train BPE tokenizer
    spm.SentencePieceTrainer.train(input=file_path, model_prefix='bpe_model', model_type='bpe', vocab_size=vocab_size)

    # Load the trained BPE tokenizer
    model_file = 'bpe_model.model'
    bpe_tokenizer = spm.SentencePieceProcessor(model_file)
    return bpe_tokenizer


### NOTE: The list 'tokenizer_list' contains list of tokenizers for Unigram and BPE, this is to be used in Q-5

In [17]:
""" This list contains list of tokenizers for Unigram and BPE, this is to be used in Question 5"""
tokenizer_list = []

### Unigram tokenizer for 1k vocab size

In [18]:
unigram_tokenizer_1k = train_unigram_tokenizer(file_path, 1000)
tokenizer_list.append(unigram_tokenizer_1k)  #to be used in Question-5

unigram_freqs, bigram_freqs, syllable_freqs, char_freqs = count_frequencies(unigram_tokenizer_1k, data)
freq_arr = [unigram_freqs, bigram_freqs, syllable_freqs, char_freqs]
write_to_file_and_print_top_k(model_name="unigram", vocab="1000", k=10, freq_arr=freq_arr)


Top 10 For unigram

र: 431118
▁के: 328920
न: 281629
।: 276966
ा: 256523
ल: 244768
▁में: 240292
▁: 233755
▁है: 215655
ी: 211097

Top 10 For bigram

('▁है', '।'): 89363
('▁के', '▁लिए'): 43425
('▁हैं', '।'): 33632
('▁है', '.'): 25838
('▁है', '▁कि'): 25125
('ों', '▁के'): 21724
('र', 'ा'): 19993
('ों', '▁में'): 19869
('▁है', ','): 19551
('▁के', '▁साथ'): 18346

Top 10 For syllables

('र', '▁'): 688273
('के', '▁'): 372533
('▁', 'प'): 350037
('▁', 'के'): 328933
('▁', 'क'): 323331
('न', '▁'): 298802
('ने', '▁'): 297998
('▁', 'स'): 261329
('्', 'र'): 260462
('ई', '▁'): 257373

Top 10 For char

('अ', '▁'): 2914975
('र्', 'अ'): 1710701
('अ', '्'): 1382365
('ए', '▁'): 1357458
('▁', 'क'): 1089979
('ई', '▁'): 1027451
('आ', '▁'): 984414
('क्', 'अ'): 852253
('स्', 'अ'): 768895
('न्', 'अ'): 705064


### Unigram tokenizer for 2k vocab size

In [19]:
unigram_tokenizer_2k = train_unigram_tokenizer(file_path, 2000)
tokenizer_list.append(unigram_tokenizer_2k)  #to be used in Question-5

unigram_freqs, bigram_freqs, syllable_freqs, char_freqs = count_frequencies(unigram_tokenizer_2k, data)
freq_arr = [unigram_freqs, bigram_freqs, syllable_freqs, char_freqs]
write_to_file_and_print_top_k(model_name="unigram", vocab="2000", k=10, freq_arr=freq_arr)


Top 10 For unigram

▁के: 324723
।: 273666
▁में: 240292
▁है: 215632
▁की: 196738
,: 190026
ी: 173237
▁को: 154736
▁से: 151426
ल: 146463

Top 10 For bigram

('▁है', '।'): 89363
('▁के', '▁लिए'): 43425
('▁हैं', '।'): 33632
('▁है', '.'): 25841
('▁है', '▁कि'): 25121
('▁है', ','): 19551
('ों', '▁के'): 19271
('▁के', '▁साथ'): 18346
('ों', '▁में'): 17781
('▁कहा', '▁कि'): 16097

Top 10 For syllables

('र', '▁'): 688273
('के', '▁'): 371528
('▁', 'प'): 328908
('▁', 'के'): 328435
('▁', 'क'): 319830
('न', '▁'): 298802
('ने', '▁'): 297206
('में', '▁'): 253766
('▁', 'स'): 253421
('▁', 'अ'): 244244

Top 10 For char

('अ', '▁'): 2914975
('र्', 'अ'): 1586357
('अ', '्'): 1382365
('ए', '▁'): 1357458
('▁', 'क'): 1093480
('ई', '▁'): 1027451
('आ', '▁'): 984414
('क्', 'अ'): 838783
('स्', 'अ'): 741573
('अं', '▁'): 700936


### BPE tokenizer for 1k vocab size

In [20]:
bpe_tokenizer_1k = train_bpe_tokenizer(file_path, 1000)
tokenizer_list.append(bpe_tokenizer_1k)  #to be used in Question-5

unigram_freqs, bigram_freqs, syllable_freqs, char_freqs = count_frequencies(bpe_tokenizer_1k, data)
freq_arr = [unigram_freqs, bigram_freqs, syllable_freqs, char_freqs]
write_to_file_and_print_top_k(model_name="BPE", vocab="1000", k=10, freq_arr=freq_arr)


Top 10 For unigram

▁के: 328806
।: 270047
▁में: 240292
▁है: 215519
▁की: 199499
,: 186960
▁को: 162462
ल: 158685
न: 156906
▁से: 152622

Top 10 For bigram

('▁है', '।'): 89363
('▁के', '▁लिए'): 43425
('▁हैं', '।'): 33632
('▁है', '.'): 25579
('▁है', '▁कि'): 25130
('▁है', ','): 19551
('▁के', '▁साथ'): 18346
('ों', '▁के'): 17977
('ने', '▁के'): 16651
('▁कहा', '▁कि'): 16100

Top 10 For syllables

('र', '▁'): 688273
('के', '▁'): 373261
('▁', 'प'): 370443
('▁', 'क'): 338393
('▁', 'के'): 328820
('ने', '▁'): 300414
('न', '▁'): 298802
('▁', 'स'): 265313
('में', '▁'): 252023
('्', 'र'): 244352

Top 10 For char

('अ', '▁'): 2914975
('र्', 'अ'): 1658587
('अ', '्'): 1382365
('ए', '▁'): 1357458
('▁', 'क'): 1074917
('ई', '▁'): 1027451
('आ', '▁'): 984414
('क्', 'अ'): 869382
('स्', 'अ'): 758394
('अं', '▁'): 700936


### BPE tokenizer for 2k vocab size

In [21]:
bpe_tokenizer_2k = train_bpe_tokenizer(file_path, 2000)
tokenizer_list.append(bpe_tokenizer_2k) #To be used in Question-5

unigram_freqs, bigram_freqs, syllable_freqs, char_freqs = count_frequencies(bpe_tokenizer_2k, data)
freq_arr = [unigram_freqs, bigram_freqs, syllable_freqs, char_freqs]
write_to_file_and_print_top_k(model_name="BPE", vocab="2000", k=10, freq_arr=freq_arr)


Top 10 For unigram

▁के: 326040
।: 270047
▁में: 240292
▁है: 215519
▁की: 197474
,: 185205
▁को: 156151
▁से: 147346
▁का: 123398
.: 116875

Top 10 For bigram

('▁है', '।'): 89363
('▁के', '▁लिए'): 43425
('▁हैं', '।'): 33632
('▁है', '.'): 25579
('▁है', '▁कि'): 25108
('▁है', ','): 19551
('▁के', '▁साथ'): 18346
('▁कहा', '▁कि'): 16096
('▁के', '▁बाद'): 14451
('ों', '▁के'): 13475

Top 10 For syllables

('र', '▁'): 688273
('के', '▁'): 373261
('▁', 'प'): 338511
('▁', 'के'): 328820
('▁', 'क'): 318280
('ने', '▁'): 300414
('न', '▁'): 298802
('▁', 'स'): 257869
('में', '▁'): 254647
('▁', 'अ'): 244244

Top 10 For char

('अ', '▁'): 2914975
('र्', 'अ'): 1580578
('अ', '्'): 1382365
('ए', '▁'): 1357458
('▁', 'क'): 1095257
('ई', '▁'): 1027451
('आ', '▁'): 984414
('क्', 'अ'): 821155
('स्', 'अ'): 744886
('अं', '▁'): 700936


## mBERT and IndicBERT Models

###  function definations for counting freqs( only for BERT models) and to get pretrained BERT tokenizer

In [22]:
from transformers import BertTokenizer, BertModel
import tensorflow as tf

def count_frequencies_bert(tokens):
    """
    Count the frequencies of unigrams, bigrams, syllable bigrams, and character bigrams in the given list of tokens.

    Parameters:
    - tokens (List[str]): A list of tokens for frequency analysis.

    Returns:
    tuple: A tuple containing four Counter objects representing the frequencies of unigrams, bigrams,
           syllable bigrams, and character bigrams in the given list of tokens.
    """
    # Count unigram frequencies
    unigram_freqs = Counter(tokens)
    
    bigrams = [(tokens[i], tokens[i + 1]) for i in range(len(tokens) - 1)]
    bigram_freqs = Counter(bigrams)

    syllables = []
    characters = []
    for tok in tokens:
        syllables.extend(syllable_convert(tok))
        characters.extend(character_conversion(tok))
        
    bigram_syllables = [(syllables[i], syllables[i + 1]) for i in range(len(syllables) - 1)]
    bigram_characters = [(characters[i], characters[i + 1]) for i in range(len(characters) - 1)]
    
    syllable_freqs = Counter(bigram_syllables)
    char_freqs = Counter(bigram_characters)

    return unigram_freqs, bigram_freqs, syllable_freqs, char_freqs


def trained_mBERT_tokenizer():
    """
    Load and return a pre-trained mBERT (multilingual BERT) tokenizer.

    Returns:
    BertTokenizer: A pre-trained mBERT tokenizer.
    """
    # Load mBERT tokenizer
    return BertTokenizer.from_pretrained('bert-base-multilingual-cased')

### mBERT tokenizer for 1k max-length

In [23]:
tokenizer = trained_mBERT_tokenizer()

# Tokenize the data
mBert_encodings = tokenizer(data, max_length=1000, truncation=True)
mBert_tokens = tokenizer.convert_ids_to_tokens(mBert_encodings['input_ids'])

unigram_freqs, bigram_freqs, syllable_freqs, char_freqs = count_frequencies_bert(mBert_tokens)
freq_arr = [unigram_freqs, bigram_freqs, syllable_freqs, char_freqs]
 
""" NOTE: vocab indicates the max-length"""
write_to_file_and_print_top_k(model_name="mBERT", vocab="1000", k=10, freq_arr=freq_arr)


Top 10 For unigram

के: 22
में: 20
।: 18
है: 15
ब: 15
##प: 14
आ: 13
,: 13
को: 13
##ी: 12

Top 10 For bigram

('आ', '##प'): 7
('ख', '##ोज'): 7
('है', '।'): 6
('है', 'कि'): 4
('हैं', ','): 4
('##ोज', 'शब्द'): 4
('##ों', 'की'): 3
('##्', '##प'): 3
('को', 'ब'): 3
('##रल', '##ैंड'): 3

Top 10 For syllables

('#', '#'): 461
('#', '्'): 51
('#', 'इ'): 40
('र', '#'): 31
('#', 'ए'): 29
('#', 'आ'): 21
('#', 'ओ'): 21
('स', '#'): 20
('क', '#'): 20
('आ', '#'): 19

Top 10 For char

('#', '#'): 460
('अ', '#'): 299
('र्', 'अ'): 98
('क्', 'अ'): 70
('आ', '#'): 52
('#', '्'): 51
('न्', 'अ'): 50
('स्', 'अ'): 49
('अ', '्'): 49
('प्', 'अ'): 42


### mBERT tokenizer for 2k max-length

In [24]:
tokenizer = trained_mBERT_tokenizer()
# Tokenize the data
mBert_encodings = tokenizer(data, max_length=2000, truncation=True)
mBert_tokens = tokenizer.convert_ids_to_tokens(mBert_encodings['input_ids'])

unigram_freqs, bigram_freqs, syllable_freqs, char_freqs = count_frequencies_bert(mBert_tokens)
freq_arr = [unigram_freqs, bigram_freqs, syllable_freqs, char_freqs]

""" NOTE: vocab indicates the max-length"""
write_to_file_and_print_top_k(model_name="mBERT", vocab="2000", k=10, freq_arr=freq_arr)


Top 10 For unigram

।: 37
में: 36
के: 34
है: 29
ब: 27
##र: 26
की: 25
##ी: 23
,: 23
ने: 22

Top 10 For bigram

('है', '।'): 12
('आ', '##प'): 7
('हैं', ','): 7
('ख', '##ोज'): 7
('हैं', '।'): 6
('है', 'कि'): 5
('के', 'लिए'): 5
('स', '##्व'): 4
('##ोज', 'शब्द'): 4
('ने', 'कहा'): 4

Top 10 For syllables

('#', '#'): 932
('#', '्'): 85
('#', 'इ'): 73
('र', '#'): 73
('#', 'आ'): 52
('#', 'ए'): 44
('स', '#'): 43
('#', 'ओ'): 38
('म', '#'): 36
('प', '#'): 33

Top 10 For char

('#', '#'): 930
('अ', '#'): 600
('र्', 'अ'): 206
('क्', 'अ'): 127
('स्', 'अ'): 102
('आ', '#'): 101
('अ', '्'): 99
('न्', 'अ'): 90
('#', '्'): 85
('प्', 'अ'): 80


### Loading pretrained IndicBERT tokenizer


In [25]:
import transformers
import collections

# Load pretrained IndicBERT Tokenizer
def IndicBERT_tokenizer():
    indicbert_model_name = "ai4bharat/indic-bert"  
    indicbert_tokenizer = transformers.AutoTokenizer.from_pretrained(indicbert_model_name)
    return indicbert_tokenizer

### IndicBERT tokenizer for 1k max-length

In [26]:
indicbert_tokenizer = IndicBERT_tokenizer()

ibert_tokens = indicbert_tokenizer(data, max_length=1000, truncation=True)
indicbert_tokens = indicbert_tokenizer.convert_ids_to_tokens(ibert_tokens['input_ids'])

unigram_freqs, bigram_freqs, syllable_freqs, char_freqs = count_frequencies_bert(indicbert_tokens)
freq_arr = [unigram_freqs, bigram_freqs, syllable_freqs, char_freqs]

""" NOTE: vocab indicates the max-length"""
write_to_file_and_print_top_k(model_name="IndicBERT", vocab="1000", k=10, freq_arr=freq_arr)


Top 10 For unigram

▁क: 67
▁ह: 38
▁म: 24
।: 23
य: 23
क: 23
▁: 20
त: 19
न: 16
स: 15

Top 10 For bigram

('▁ह', '।'): 11
('▁ह', ','): 8
('▁क', '▁'): 6
('त', '▁ह'): 6
('▁ल', 'ए'): 5
('न', '▁क'): 5
('▁आप', 'क'): 4
('▁द', 'य'): 4
('य', '▁ह'): 4
('▁ह', '▁क'): 4

Top 10 For syllables

('क', '▁'): 99
('▁', 'क'): 98
('न', '▁'): 60
('र', '▁'): 60
('▁', 'ह'): 53
('ह', '▁'): 50
('म', '▁'): 40
('▁', 'स'): 38
('▁', 'म'): 33
('स', '▁'): 32

Top 10 For char

('अ', '▁'): 547
('क्', 'अ'): 174
('र्', 'अ'): 149
('अ', 'र्'): 115
('▁', 'क्'): 98
('न्', 'अ'): 95
('स्', 'अ'): 93
('ह्', 'अ'): 87
('अ', 'न्'): 77
('ल्', 'अ'): 77


### IndicBERT tokenizer for 2k max-length

In [27]:
indicbert_tokenizer = IndicBERT_tokenizer()

ibert_tokens = indicbert_tokenizer(data, max_length=2000, truncation=True)
indicbert_tokens = indicbert_tokenizer.convert_ids_to_tokens(ibert_tokens['input_ids'])

unigram_freqs, bigram_freqs, syllable_freqs, char_freqs = count_frequencies_bert(indicbert_tokens)
freq_arr = [unigram_freqs, bigram_freqs, syllable_freqs, char_freqs]

""" NOTE: vocab indicates the max-length"""
write_to_file_and_print_top_k(model_name="IndicBERT", vocab="2000", k=10, freq_arr=freq_arr)


Top 10 For unigram

▁क: 131
▁ह: 72
य: 52
▁म: 48
।: 42
▁न: 41
क: 41
▁: 39
न: 38
त: 35

Top 10 For bigram

('▁ह', '।'): 22
('▁ह', ','): 14
('▁ल', 'ए'): 10
('त', '▁ह'): 10
('▁न', 'ह'): 8
('▁रह', '▁ह'): 8
('▁क', '▁ल'): 8
('▁क', '▁'): 7
('।', '▁इस'): 7
('य', '▁क'): 7

Top 10 For syllables

('▁', 'क'): 190
('क', '▁'): 188
('र', '▁'): 120
('न', '▁'): 117
('ह', '▁'): 117
('▁', 'ह'): 92
('▁', 'स'): 83
('म', '▁'): 81
('▁', 'म'): 76
('त', '▁'): 64

Top 10 For char

('अ', '▁'): 1099
('क्', 'अ'): 347
('र्', 'अ'): 287
('अ', 'र्'): 223
('न्', 'अ'): 203
('स्', 'अ'): 194
('▁', 'क्'): 190
('ह्', 'अ'): 172
('म्', 'अ'): 152
('अ', 'न्'): 147


### WhiteSpace tokenizer

In [28]:
def whitespace_tokenize(text):
    """This function implements basic whitespace cleaning and splitting on a piece of text."""
    text = text.strip()
    if not text:
        return []
    tokens = text.split()
    return tokens

tokens = whitespace_tokenize(data)
# print(tokens)
unigram_freqs, bigram_freqs, syllable_freqs, char_freqs = count_frequencies_bert(tokens)
freq_arr = [unigram_freqs, bigram_freqs, syllable_freqs, char_freqs]

write_to_file_and_print_top_k(model_name="WhiteSpace", vocab="", k=10, freq_arr=freq_arr)


Top 10 For unigram

के: 316314
में: 237426
की: 189769
को: 145266
से: 136979
और: 114563
का: 109743
ने: 102378
पर: 88560
है।: 88260

Top 10 For bigram

('के', 'लिए'): 42448
('है', 'कि'): 24816
('के', 'साथ'): 17075
('कहा', 'कि'): 15922
('के', 'बाद'): 14163
('है', 'और'): 10096
('ने', 'कहा'): 9098
('करने', 'के'): 8826
('बताया', 'कि'): 6392
('को', 'लेकर'): 5977

Top 10 For syllables

('र', '्'): 230510
('क', 'र'): 176593
('स', '्'): 168049
('्', 'र'): 167056
('क', '्'): 157819
('प', '्'): 141260
('त', '्'): 123843
('औ', 'र'): 115750
('प', 'र'): 107644
('्', 'य'): 91066

Top 10 For char

('र्', 'अ'): 1403772
('अ', '्'): 1382370
('क्', 'अ'): 777423
('स्', 'अ'): 686835
('अ', 'र्'): 641405
('अ', 'क'): 628621
('न्', 'अ'): 599585
('प्', 'अ'): 547147
('त्', 'अ'): 480418
('क', 'ए'): 407129


## Question 5

#### The below cell will require a file named 'cs689_assignment.txt' in current directory for execution, I have added the file in zip submission

In [29]:
sentences = ""
sentences_file = 'cs689_assignment.txt'

with open(sentences_file, 'r', encoding='utf-8') as file:
    for line in file:
        if line !="\n":
            temp = line.strip()
            sentences = sentences+temp[3:]
print(sentences)

उन्होंने कहा कि मौजूदा सरकार ने पंजाब को कंगाल बनाकर रख दिया है तथा सरकार विकास कार्यों का बोझ उठाने के योग्य नहीं है जिस कारण राज्य की जनता ने अकाली-भाजपा को सिरे से नकार दिया है।भारत ने इस मामले में वोटिंग ( voting ) न करने का फैसला किया।वजन 162 ग्राममुख्य विपक्षी दल कांग्रेस ने संबित के बयान की आलोचना करते हुए कहा कि यह भाजपा का अहंकार बोल रहा है।मुझे अपने विपक्षी मित्रों के तर्क इस वैज्ञानिक के निष्‍कर्ष के समान ही प्रतीत हो रहे हैं....अगर आपको व्यापार में सफलता मिल रही है तो अपने घर के उत्तर पूर्व कोने को गंगाजल से धोकर वहां हल्दी का स्वास्तिक बनाये। फिर उसकी पूजा करे गुड़ का भोग लगाए। ऐसा करने से आपके व्यापार में सफलता बढेगी साथ ही आप जीवन में सफलता की सीढी भी चढेंगे।नीति आयोग के CEO अमिताभ कांत ने कहा कि देश में पिछले तीन साल में प्रति ग्राहक मोबाइल डाटा कंज्मप्शन में 142 फीसदी की ग्रोथ दर्ज की गई है।भारत अभी टी-20 रैंकिंग में पांचवें स्थान पर है और उसे चौथे स्थान पर पहुंचने के लिये वर्तमान सीरीज 5-0 से जीतनी होगी.लुटने के डर से व्यापारी ने दुकान की जगह थाने से बंटवाई यूरिया इनकी

### Functions for Precision, Recall and F1-Score  
(In these definations, doc string is not added because the functions are self-explanatory with relevant comment added)

In [30]:
def precision(true_tokens, pred_tokens):
    true_positives = len(set(true_tokens) & set(pred_tokens))
    false_positives = len(pred_tokens) - true_positives
    
    #case where precision is being dividing by zero
    if true_positives + false_positives == 0:
        return 0 
    
    precision_value = true_positives / (true_positives + false_positives)
    return precision_value

def recall(true_tokens, pred_tokens):
    true_positives = len(set(true_tokens) & set(pred_tokens))
    false_negatives = len(true_tokens) - true_positives
    
    #case where recall is being dividing by zero
    if true_positives + false_negatives == 0:
        return 0  
    
    recall_value = true_positives / (true_positives + false_negatives)
    return recall_value

def f1_score(true_tokens, pred_tokens):
    precision_value = precision(true_tokens, pred_tokens)
    recall_value = recall(true_tokens, pred_tokens)
    
    # Handle the case where F1-Score dividing by zero
    if precision_value + recall_value == 0:
        return 0  
    
    f1_score_value = 2 * (precision_value * recall_value) / (precision_value + recall_value)
    return f1_score_value


#### The below cell will require a file named 'ground_truth_tokens.txt'  in current directory for execution, I have added the file in zip submission
NOTE: The file downloaded from website having question-3 will need cleaning as that contains questions too but here only word-groups representing truth tokens will be needed, that's why cleaned file is attached with zip

In [31]:
file_path = 'ground_truth_tokens.txt'

with open(file_path, 'r', encoding='utf-8') as file:
    tokens = file.read().split(',')

true_tokens = [word.strip() for word in tokens if word!=" "]
print("List of True Tokens:", true_tokens[:50])


List of True Tokens: ['उन्होंने', 'कहा कि', 'मौजूदा', 'सरकार ने', 'पंजाब को', 'कंगाल', 'बनाकर', 'रख', 'दिया है', 'तथा', 'सरकार', 'विकास', 'कार्यों का', 'बोझ', 'उठाने के', 'योग्य', 'नहीं', 'है', 'जिस', 'कारण', 'राज्य की', 'जनता ने', 'अकाली-भाजपा को', 'सिरे से', 'नकार', 'दिया है।\n\n\nभारत ने', 'इस', 'मामले में', 'वोटिंग', 'न', 'करने का', 'फैसला', 'किया।\n\n\nवजन', '162 ग्राम\n\n\nमुख्य', 'विपक्षी दल', 'कांग्रेस ने', 'संबित के', 'बयान की', 'आलोचना', 'करते हुए', 'कहा कि', 'यह', 'भाजपा का', 'अहंकार', 'बोल', 'रहा है।', 'मुझे', 'अपने', 'विपक्षी', 'मित्रों के']


### Precision, Recall and F1-Score for Unigram(1K,2K) and BPE (1K,2K)

In [32]:
"""  data_file is path to file of 25 sentences
     unigram_tokenizer is tokenizer for unigram model   """

""" Tokenizer list contains the trained tokenizer (in Question 4) for Unigram and BPE"""

model_name = ["Unigram_1k","Unigram_2k","BPE_1k","BPE_2k"]
index = 0

for tokenizer in tokenizer_list:
    pred_tokens = tokenizer.encode(sentences, out_type=str)
    #  Removing '_' from staring of tokens,'_' works just as seperator so it will not affect the quality of tokens
    pred_tokens = [token[1:] if token.startswith('▁') else token for token in pred_tokens]
    
    # Precision
    precision_value = precision(true_tokens, pred_tokens)
    print(f"Precision for {model_name[index]}: {precision_value:.2f}")

    # Recall
    recall_value = recall(true_tokens, pred_tokens)
    print(f"Recall for {model_name[index]}: {recall_value:.2f}")

    # F1-score
    f1_score_value = f1_score(true_tokens, pred_tokens)
    print(f"F1-Score for {model_name[index]}: {f1_score_value:.2f}")
    
    index= index+1
    print("\n")

Precision for Unigram_1k: 0.06
Recall for Unigram_1k: 0.19
F1-Score for Unigram_1k: 0.09


Precision for Unigram_2k: 0.09
Recall for Unigram_2k: 0.25
F1-Score for Unigram_2k: 0.14


Precision for BPE_1k: 0.05
Recall for BPE_1k: 0.16
F1-Score for BPE_1k: 0.08


Precision for BPE_2k: 0.08
Recall for BPE_2k: 0.23
F1-Score for BPE_2k: 0.12




### Precision, Recall and F1-Score for mBERT

In [33]:
# NO need of tokenizer list, because pretrained tok is used everywhere
mBERT_tokenizer = trained_mBERT_tokenizer()
mBert_encodings = mBERT_tokenizer.encode_plus(sentences, max_length=1000, truncation=True, padding=True)
pred_tokens = mBERT_tokenizer.convert_ids_to_tokens(mBert_encodings['input_ids'])

# Precision
precision_value = precision(true_tokens, pred_tokens)
print(f"Precision: {precision_value:.2f}")

# Recall
recall_value = recall(true_tokens, pred_tokens)
print(f"Recall: {recall_value:.2f}")

# F1-score
f1_score_value = f1_score(true_tokens, pred_tokens)
print(f"F1-Score: {f1_score_value:.2f}")

Precision: 0.06
Recall: 0.18
F1-Score: 0.09


### Precision, Recall and F1-Score for IndicBERT

In [34]:
# NO need of tokenizer list, because pretrained tok is used everywhere
indicbert_tokenizer = IndicBERT_tokenizer()
ibert_tokens = indicbert_tokenizer.encode_plus(sentences, max_length=1000, truncation=True)
pred_tokens = indicbert_tokenizer.convert_ids_to_tokens(ibert_tokens['input_ids'])
pred_tokens = [token[1:] if token.startswith('▁') else token for token in pred_tokens]

# Precision
precision_value = precision(true_tokens, pred_tokens)
print(f"Precision: {precision_value:.2f}")

# Recall
recall_value = recall(true_tokens, pred_tokens)
print(f"Recall: {recall_value:.2f}")

# F1-score
f1_score_value = f1_score(true_tokens, pred_tokens)
print(f"F1-Score: {f1_score_value:.2f}")
print("\n")

Precision: 0.02
Recall: 0.04
F1-Score: 0.02




### Precision, Recall and F1-Score for WhiteSpace Tokenizer

In [35]:
pred_tokens = whitespace_tokenize(sentences)
# print(pred_tokens)

# Precision
precision_value = precision(true_tokens, pred_tokens)
print(f"Precision: {precision_value:.2f}")

# Recall
recall_value = recall(true_tokens, pred_tokens)
print(f"Recall: {recall_value:.2f}")

# F1-score
f1_score_value = f1_score(true_tokens, pred_tokens)
print(f"F1-Score: {f1_score_value:.2f}")

Precision: 0.36
Recall: 0.52
F1-Score: 0.42


# 

# Problem-6

### Observations from comparison:
    On assessing different tokenizers to tokenizing Hindi sentences, the whitespace tokenizer clearly excels with its high precision, demonstrating its ability to capture the structures of the hindi language which make sense because in hindi words generally makes more sense when tokenized by space as delimeter . Unigram and BPE tokenizers also exhibit moderate performance, while mBERT and IndicBERT tokenizers perform more or less similarly. The prominence of whitespace tokenization highlights that, despite advancements in tokenization methods, the use of whitespace remains effective in accurately representing the Hindi language processing. This suggests that while tokenizers are valuable tools, there is still room for growth and whitespace tokenization still remains a reliable option for working with Hindi text.